In [1]:
from google.colab import drive

drive.mount('/content/drive/')

ModuleNotFoundError: No module named 'google.colab'

In [1]:
import pandas as pd

In [2]:
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType, FloatType
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('ML_Project').setMaster('local').set("spark.executor.memory", "6g")
sc = pyspark.SparkContext(conf=conf)
#Create SparkSession
spark = SparkSession(sc)

In [89]:
# import collision_road matching data with segment IDs
df_collision_segment_pairs = pd.read_csv('../data/output/collision_segment_pairs.csv')

# import collision data
df_collisions = pd.read_csv('../data/output/collisions.csv')

# load all segment_ID in Berlin
df_segments = pd.read_csv('../data/output/road_segments.csv')


c:\users\bened\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (5,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [90]:
# Retain only segment IDs from full segment list
df_segment_ids = df_segments[['segment_id']]
df_segment_ids

,segment_id
0,0
1,1
2,2
3,3
4,4
...,...
43105,43105
43106,43106
43107,43107
43108,43108


In [5]:
# Merge Collision Segment pairs with collision data set to get positive sample
df_positive_sample = df_collision_segment_pairs.merge(df_collisions, left_on='col_id', right_on='col_id', suffixes=('','_y'), copy=False)
df_positive_sample = df_positive_sample.drop(['street_y', 'year_y', 'month_y', 'hour_y',
       'weekday_y'], axis=1)
df_positive_sample.columns

Index(['col_id', 'street', 'year', 'month', 'weekday', 'hour', 'segment_id',
       'bezirk', 'stadtteil', 'strassenna', 'objectid', 'land', 'district',
       'LOR', 'LOR_ab_2021', 'acc_cat', 'acc_type1', 'acc_type2', 'lightratio',
       'bike', 'car', 'foot', 'motor', 'vehicle', 'other', 'road_con',
       'LINREFX', 'LINREFY', 'XGCSWGS84', 'YGCSWGS84'],
      dtype='object')

In [6]:
# Some type conversions
df_positive_sample.bezirk = df_positive_sample.bezirk.astype(str)
df_positive_sample.stadtteil = df_positive_sample.stadtteil.astype(str)
df_positive_sample.strassenna = df_positive_sample.strassenna.astype(str)
df_positive_sample.other = df_positive_sample.other.astype(str)
df_positive_sample.street = df_positive_sample.street.astype(str)
df_positive_sample.road_con = df_positive_sample.road_con.astype(str)
df_positive_sample

,col_id,street,year,month,weekday,hour,segment_id,bezirk,stadtteil,strassenna,...,car,foot,motor,vehicle,other,road_con,LINREFX,LINREFY,XGCSWGS84,YGCSWGS84
0,0,Samariterviertel,2018,1,4,15,6209.0,Friedrichshain-Kreuzberg,Friedrichshain,Frankfurter Allee,...,1,1,0,0,0,1,"803604,1562","5827580,738","13,4750178","52,51359681"
1,1,Ziekowstraße/Freie Scholle,2018,1,2,11,41374.0,Reinickendorf,Tegel,A 111 BAB Autobahnzubringer Hamburg,...,1,0,0,0,0,0,"790639,5854","5835009,589","13,29102205","52,58725906"
2,1,Ziekowstraße/Freie Scholle,2018,1,2,11,41373.0,Reinickendorf,Tegel,A 111 BAB Autobahnzubringer Hamburg,...,1,0,0,0,0,0,"790639,5854","5835009,589","13,29102205","52,58725906"
3,2,Barnimkiez,2018,1,3,9,4410.0,Pankow,Prenzlauer Berg,Otto-Braun-Straße,...,1,1,0,0,0,0,"799827,9618","5828733,523","13,42057818","52,52601854"
4,3,Volkspark (Rudolf-Wilde-Park),2018,1,2,17,1452.0,Tempelhof-Schöneberg,Schöneberg,Hauptstraße,...,1,1,0,0,0,0,"795223,509","5823525,402","13,34828776","52,48184447"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96661,38850,nan,2020,12,3,16,11485.0,Mitte,Mitte,Leipziger Straße,...,1,1,0,0,0,1,"797982,8966","5826897,222","13,39181741","52,51057"
96662,38850,nan,2020,12,3,16,11486.0,Mitte,Mitte,Charlottenstraße,...,1,1,0,0,0,1,"797982,8966","5826897,222","13,39181741","52,51057"
96663,38850,nan,2020,12,3,16,26570.0,Mitte,Mitte,Charlottenstraße,...,1,1,0,0,0,1,"797982,8966","5826897,222","13,39181741","52,51057"
96664,38850,nan,2020,12,3,16,11484.0,Mitte,Mitte,Leipziger Straße,...,1,1,0,0,0,1,"797982,8966","5826897,222","13,39181741","52,51057"


In [7]:
# generate all dates from start to end
import time
import datetime



start = "31/12/2017"
end = "31/12/2020"
date = datetime.datetime.strptime(start, "%d/%m/%Y")

dates = list()
while(date != datetime.datetime.strptime(end, "%d/%m/%Y")): 
    date += datetime.timedelta(days=1)
    for i in range(24):
        dates.append((date.strftime("%Y-%m-%d"), i))

In [84]:
# Ge date hour pairs and preprocess them to fit with formatting of collision data (e.g. Sunday: 1)
df_date_hour_pairs = pd.DataFrame(dates, columns = ['date', 'hour'])
df_date_hour_pairs['date'] = pd.to_datetime(df_date_hour_pairs['date'])
df_date_hour_pairs['year'] = df_date_hour_pairs['date'].dt.year
df_date_hour_pairs['month'] = df_date_hour_pairs['date'].dt.month
df_date_hour_pairs['weekday'] = df_date_hour_pairs['date'].dt.weekday + 2
df_date_hour_pairs.loc[(df_date_hour_pairs.weekday == 8),'weekday']=1
df_date_hour_pairs = df_date_hour_pairs.drop('date', axis=1)
df_date_hour_pairs = df_date_hour_pairs.drop_duplicates()
df_date_hour_pairs

,hour,year,month,weekday
0,0,2018,1,2
1,1,2018,1,2
2,2,2018,1,2
3,3,2018,1,2
4,4,2018,1,2
...,...,...,...,...
25723,19,2020,12,2
25724,20,2020,12,2
25725,21,2020,12,2
25726,22,2020,12,2


In [9]:
df_date_hour_pairs = spark.createDataFrame(df_date_hour_pairs) 
df_segments = spark.createDataFrame(df_segments) 
df_positive_sample = spark.createDataFrame(df_positive_sample) 

In [10]:
# Create all possible year, month, day, segment combinations for 2018-2020
df_combinations = df_date_hour_pairs.crossJoin(df_segments).repartition(100)
df_combinations.count()

260729280

In [14]:
# Drop all accident time paris that acutally have been recorded as positive examples (collisions occurred)
df_neg_combinations = df_combinations.join(df_positive_sample,(df_combinations.year == df_positive_sample.year) & 
                                                            (df_combinations.month == df_positive_sample.month) & 
                                                            (df_combinations.weekday == df_positive_sample.weekday) & 
                                                            (df_combinations.hour == df_positive_sample.hour) &
                                                            (df_combinations.segment_id == df_positive_sample.segment_id), 
                                       "leftanti").repartition(100)
df_neg_combinations.count()

260632891

In [15]:
# Draw 0.1% uniform sample of negative combinations to replicate Hebert approach
df_neg_combinations = df_neg_combinations.sample(False, 0.001, seed=None)
df_neg_combinations.count()

1643427

In [23]:
# Convert it to Pandas DF
df_neg_combinations = df_neg_combinations.toPandas()

,col_id,street,year,month,weekday,hour,segment_id,bezirk,stadtteil,strassenna,...,car,foot,motor,vehicle,other,road_con,LINREFX,LINREFY,XGCSWGS84,YGCSWGS84
0,0,Samariterviertel,2018,1,4,15,6209.0,Friedrichshain-Kreuzberg,Friedrichshain,Frankfurter Allee,...,1,1,0,0,0,1,"803604,1562","5827580,738","13,4750178","52,51359681"
1,1,Ziekowstraße/Freie Scholle,2018,1,2,11,41374.0,Reinickendorf,Tegel,A 111 BAB Autobahnzubringer Hamburg,...,1,0,0,0,0,0,"790639,5854","5835009,589","13,29102205","52,58725906"
2,1,Ziekowstraße/Freie Scholle,2018,1,2,11,41373.0,Reinickendorf,Tegel,A 111 BAB Autobahnzubringer Hamburg,...,1,0,0,0,0,0,"790639,5854","5835009,589","13,29102205","52,58725906"
3,2,Barnimkiez,2018,1,3,9,4410.0,Pankow,Prenzlauer Berg,Otto-Braun-Straße,...,1,1,0,0,0,0,"799827,9618","5828733,523","13,42057818","52,52601854"
4,3,Volkspark (Rudolf-Wilde-Park),2018,1,2,17,1452.0,Tempelhof-Schöneberg,Schöneberg,Hauptstraße,...,1,1,0,0,0,0,"795223,509","5823525,402","13,34828776","52,48184447"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96661,38850,nan,2020,12,3,16,11485.0,Mitte,Mitte,Leipziger Straße,...,1,1,0,0,0,1,"797982,8966","5826897,222","13,39181741","52,51057"
96662,38850,nan,2020,12,3,16,11486.0,Mitte,Mitte,Charlottenstraße,...,1,1,0,0,0,1,"797982,8966","5826897,222","13,39181741","52,51057"
96663,38850,nan,2020,12,3,16,26570.0,Mitte,Mitte,Charlottenstraße,...,1,1,0,0,0,1,"797982,8966","5826897,222","13,39181741","52,51057"
96664,38850,nan,2020,12,3,16,11484.0,Mitte,Mitte,Leipziger Straße,...,1,1,0,0,0,1,"797982,8966","5826897,222","13,39181741","52,51057"


In [85]:
df_negative_sample = df_neg_combinations.merge(df_positive_sample.drop_duplicates('segment_id'), how="left", left_on=["segment_id"], right_on=["segment_id"],suffixes=('','_del')).drop(['year_del','month_del','hour_del','weekday_del'],axis=1)

In [92]:
# Generate new outcome column: "0" means non collision has occurred
df_negative_sample['collision'] = 0
df_positive_sample['collision'] = 1

In [95]:
# Add midpoint (geographic centers) of segments
df_negative_sample = df_negative_sample.merge(df_segments.drop_duplicates('segment_id')[['segment_id','mid_lat', 'mid_lon', 'length_m', 'side_strt']], how="left", left_on="segment_id", right_on="segment_id")
df_positive_sample = df_positive_sample.merge(df_segments.drop_duplicates('segment_id')[['segment_id','mid_lat', 'mid_lon', 'length_m', 'side_strt']], how="left", left_on="segment_id", right_on="segment_id")

In [ ]:
# EXPORT
df_negative_sample.to_csv('../data/output/negative_sample.csv')
df_positive_sample.to_csv('../data/output/positive_sample.csv')